In [1]:
import os
import pandas as pd
import time
import pickle
import csv

In [2]:
pkl_file = '../ppi_ml/results/elutions/pkl/leca.unique.elut.ordered.pkl'
pkl_file_norm = '../ppi_ml/results/elutions/pkl/leca.unique.norm.elut.ordered.pkl'
psm_cutoff = 150
leca_ogs = '../ppi_ml/annotations/leca_euNOGs_annotated.csv'
outfile = f'../ppi_ml/results/elutions/elut_ordered/leca.unique.{psm_cutoff}p.elut.ordered'
outfile_leca = f'../ppi_ml/results/elutions/elut_ordered/leca.unique.filtdollo.{psm_cutoff}p.elut.ordered'
outfile_norm = f'../ppi_ml/results/elutions/elut_ordered/leca.unique.{psm_cutoff}p.norm.elut.ordered'
outfile_norm_leca = f'../ppi_ml/results/elutions/elut_ordered/leca.unique.filtdollo.{psm_cutoff}p.norm.elut.ordered'
id_outfile = f'../ppi_ml/results/elutions/id_filters/{psm_cutoff}p_ids.txt'

In [3]:
t0 = time.time()
df = pd.read_pickle(pkl_file)
print(f"Pickle loading time = {time.time() - t0}")

Pickle loading time = 6.65683913230896


In [4]:
df.set_index('orthogroup', inplace=True)
print(f'Input data shape = {df.shape}')

Input data shape = (163763, 10481)


In [5]:
print(f'Summing PSMs for {df.shape[0]} IDs and {df.shape[1]} fractions ... ')
t0 = time.time()
df['sum'] = df.sum(axis=1)
df['sum'].sort_values(ascending=False)
print(f'Time to completion = {time.time() - t0}')

Summing PSMs for 163763 IDs and 10481 fractions ... 
Time to completion = 5.332030773162842


In [6]:
print(f'Filtering IDs with >= {psm_cutoff} total PSMs ...')
fdf = df[df['sum'] >= psm_cutoff]
print(f'{len(fdf)}/{df.shape[0]} IDs have >= {psm_cutoff} total PSMs.')

Filtering IDs with >= 150 total PSMs ...
17868/163763 IDs have >= 150 total PSMs.


In [7]:
vals = [x for x in df['sum'] if x >= psm_cutoff]
print(f'Range = ({min(vals)}, {max(vals)})')
print(f'# IDs = {len(vals)}')

Range = (150.0, 585526.0)
# IDs = 17868


In [8]:
fdf.reset_index(inplace=True)
del fdf['sum']
fdf.head()

,orthogroup,Rotifer_SEC_09a_04162018,Rotifer_SEC_10a_04162018,Rotifer_SEC_11a_04162018,Rotifer_SEC_12a_04162018,Rotifer_SEC_13a_04142018,Rotifer_SEC_14a_04142018,Rotifer_SEC_15a_04142018,Rotifer_SEC_16a_04142018,Rotifer_SEC_17a_04152018,...,Wheatgrass_SEC_40a_02022017,Wheatgrass_SEC_41a_02022017,Wheatgrass_SEC_42a_02022017,Wheatgrass_SEC_43a_02022017,Wheatgrass_SEC_44a_02032017,Wheatgrass_SEC_45a_02032017,Wheatgrass_SEC_47b_02032017_170207030854,Wheatgrass_SEC_48a_02032017,Wheatgrass_SEC_49a_02032017,Wheatgrass_SEC_50a_02032017
0,ENOG502QPJE,0.0,0.0,0.0,4.0,2.0,1.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ENOG502QPK4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ENOG502QPKE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENOG502QPKJ,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ENOG502QPKV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
fdf.to_csv(outfile, index=False)
fdf.to_pickle(outfile+'.pkl')

In [10]:
fdf['orthogroup'].head()

0    ENOG502QPJE
1    ENOG502QPK4
2    ENOG502QPKE
3    ENOG502QPKJ
4    ENOG502QPKV
Name: orthogroup, dtype: object

In [11]:
with open(id_outfile, 'w') as f:
    f.write('\n'.join(fdf['orthogroup']))

In [12]:
ndf = pd.read_pickle(pkl_file_norm)
nfdf = ndf[ndf['orthogroup'].isin(fdf['orthogroup'])]
nfdf.head()

,orthogroup,Rotifer_SEC_09a_04162018,Rotifer_SEC_10a_04162018,Rotifer_SEC_11a_04162018,Rotifer_SEC_12a_04162018,Rotifer_SEC_13a_04142018,Rotifer_SEC_14a_04142018,Rotifer_SEC_15a_04142018,Rotifer_SEC_16a_04142018,Rotifer_SEC_17a_04152018,...,Wheatgrass_SEC_40a_02022017,Wheatgrass_SEC_41a_02022017,Wheatgrass_SEC_42a_02022017,Wheatgrass_SEC_43a_02022017,Wheatgrass_SEC_44a_02032017,Wheatgrass_SEC_45a_02032017,Wheatgrass_SEC_47b_02032017_170207030854,Wheatgrass_SEC_48a_02032017,Wheatgrass_SEC_49a_02032017,Wheatgrass_SEC_50a_02032017
0,ENOG502QPJE,0.0000,0.0,0.0,1.0,0.5,0.25,0.75,0.75,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ENOG502QPK4,0.0000,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ENOG502QPKE,0.0000,0.0,0.0,0.0,0.0,1.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENOG502QPKJ,0.0625,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ENOG502QPKV,0.0000,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
nfdf.to_csv(outfile_norm, index=False)
nfdf.to_pickle(outfile_norm+'.pkl')

In [14]:
ldf = pd.read_csv(leca_ogs)
ldf.head()

,level_2759,level_1,num_rootNOGs,gene_fusion_product,human_entry,human_gene_names_primary,human_protein_names,human_function_cc,human_subcellular_location_cc,human_gene_ontology_biological_process,human_gene_ontology_molecular_function,arath_entry,arath_gene_names_primary,arath_protein_names,arath_function_cc,arath_subcellular_location_cc,arath_gene_ontology_biological_process,arath_gene_ontology_molecular_function
0,ENOG502QTMI,COG4294,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ENOG502RHDM,COG1670,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENOG502S5MP,COG0666,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KOG3656,"KOG3714,KOG2579,KOG1057,KOG0500,KOG1548,KOG422...",76,True,"A6NET4, A6NMS3, O00155, O00270, O00421, O00590...","OR5K3, OR5K4, GPR25, GPR31, CCRL2, ACKR2, RRH,...","Olfactory receptor 5K3, Olfactory receptor 5K4...","FUNCTION: Odorant receptor. {ECO:0000305}., FU...",SUBCELLULAR LOCATION: Cell membrane; Multi-pas...,"NA, G protein-coupled receptor signaling pathw...",G protein-coupled receptor activity [GO:000493...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KOG3657,"COG5128,KOG0937,KOG2595,COG5078,COG2272,COG074...",9,True,P54098,POLG,DNA polymerase subunit gamma-1 (EC 2.7.7.7) (M...,FUNCTION: Involved in the replication of mitoc...,SUBCELLULAR LOCATION: Mitochondrion {ECO:00002...,"aging [GO:0007568]; base-excision repair, gap-...",3'-5' exonuclease activity [GO:0008408]; chrom...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
lnfdf = nfdf[nfdf['orthogroup'].isin(ldf['level_2759'])]
lnfdf.head()
lnfdf.to_csv(outfile_norm_leca, index=False)
lnfdf.to_pickle(outfile_norm_leca+'.pkl')

In [16]:
lfdf = fdf[fdf['orthogroup'].isin(ldf['level_2759'])]
lfdf.head()
lfdf.to_csv(outfile_leca, index=False)
lfdf.to_pickle(outfile_leca+'.pkl')

In [ ]:
cutoff_dict = dict()
for i in range(0, 501):
    psm_cutoff = i
    vals = [x for x in df['sum'] if x >= psm_cutoff]
    num_vals = len(vals)
    cutoff_dict.update({i: num_vals})

In [ ]:
header = ['cutoff','num_ids']
with open('../ppi_ml/results/elutions/counts/cutoff_counts.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for cutoff, vals in cutoff_dict.items():
        writer.writerow([cutoff, vals])

In [ ]:
del df
del fdf
del ndf
del nfdf